In [ ]:
# Preperare distributed
using Distributed
addprocs(Sys.CPU_THREADS - nprocs() - 1)
print("Number of workers: ", nprocs(), "\nNumber of CPU threads: ", Sys.CPU_THREADS, "\n")
using Base.Threads
print("Number of threads: ", Threads.nthreads(), "\n") # Check number of threads available

using ProgressMeter

# Benchmarking
using BenchmarkTools

# Helpers
include("helpers/diffusion.jl")

# Plotting
@everywhere using Plots
using LaTeXStrings
include("helpers/distributed_gif.jl")


In [ ]:
N = 50
L = 1.0
dx = L / N
D = 1.0
dt = 0.00001
c_0 = zeros(N, N)
c_0[:, end] .= 1.0

In [ ]:
# Plotting parameters
ticks = (1:N/4:N+1, 0:L/4:L)
lims = (1, N + 1)
heatmap_kwargs = Dict(
    :aspect_ratio => 1,
    :xlabel => "x",
    :ylabel => "y",
    :xticks => ticks,
    :yticks => ticks,
    :xlims => lims,
    :ylims => lims,
    :dpi => 150
)

heatmap_kwargs
heatmap(c_0', title="Initial condition"; heatmap_kwargs...)

In [ ]:
do_bench = true

In [ ]:
if do_bench
    @benchmark c_next_jacobi($c_0)
end

In [ ]:
if do_bench
    _c_0 = copy(c_0)
    @benchmark c_next_gauss_seidel!($_c_0)
end

In [ ]:
if do_bench
    _c_0 = copy(c_0)
    @benchmark c_next_SOR!($_c_0)
end

## 1.6 H

In [ ]:
c_JACOBI = copy(c_0)
c_GAUSS_SEIDEL = copy(c_0)
c_SOR = copy(c_0)

k = 100
for i in 1:k
    c_JACOBI = c_next_jacobi(c_JACOBI)
    c_next_gauss_seidel!(c_GAUSS_SEIDEL)
    c_next_SOR!(c_SOR)
end

analytical_sol = (t) -> [c_anal(x, t, 1.0) for x in LinRange(0, L, N)]

plot_k_100 = plot(analytical_sol(1.0), title=L"Solution at $k_i= %$k$", xlabel="y", ylabel="c(y)", linestyle=:dot, label="Analytical", dpi=150)
plot!(c_JACOBI[1, :], label="Jacobi")
plot!(c_GAUSS_SEIDEL[1, :], label="Gauss-Seidel")
display(plot!(c_SOR[1, :], label="SOR"))
savefig("plots/diffusion_y_axis_k_100.png")

In [ ]:
tol = 1e-6

In [ ]:
c_JACOBI, _deltas = solve_until_tol(c_next_jacobi, copy(c_0), tol, 10_000)
c_GAUSS_SEIDEL, _deltas = solve_until_tol(c_next_gauss_seidel!, copy(c_0), tol, 10_000)
c_SOR, _deltas = solve_until_tol(c_next_SOR!, copy(c_0), tol, 10_000)

plot(c_JACOBI[1, :]- analytical_sol(1.0) , title="Error at convergence", xlabel="y", ylabel="Error", linestyle=:dot, label="Analytical - Jacobi", dpi=150)#, yscale=:log10, ylims=(1e-6, 1e-4))
plot!(c_GAUSS_SEIDEL[1, :]- analytical_sol(1.0) , label="Analytical - Gauss-Seidel")
display(plot!(c_SOR[1, :]- analytical_sol(1.0) , label="Analytical - SOR"))
savefig("plots/diffusion_error_y_axis.png")

## 1.6 I

In [ ]:
# Get delta for each of the methods as a function of iterations

deltas_JACOBI::Vector{Float64}  = []
deltas_GAUSS_SEIDEL::Vector{Float64} = []
deltas_SOR_199::Vector{Float64} = []
deltas_SOR_193::Vector{Float64} = []
deltas_SOR_185::Vector{Float64} = []

c_old_j = copy(c_0)
c_new_gs = copy(c_0)
c_new_sor_199 = copy(c_0)
c_new_sor_193 = copy(c_0)
c_new_sor_185 = copy(c_0)

# Run until all methods have converged, allow each method equally many iterations
while true
    # Jacobi
    c_new_j = c_next_jacobi(c_old_j)
    delta_j = delta(c_old_j, c_new_j)
    c_old_j = c_new_j
    push!(deltas_JACOBI, delta_j)

    # Gauss-Seidel
    c_old_gs = copy(c_new_gs)
    c_next_gauss_seidel!(c_new_gs)
    delta_gs = delta(c_old_gs, c_new_gs)
    push!(deltas_GAUSS_SEIDEL, delta_gs)

    # SOR
    c_old_sor_199 = copy(c_new_sor_199)
    c_old_sor_193 = copy(c_new_sor_193)
    c_old_sor_185 = copy(c_new_sor_185)
    c_next_SOR!(c_new_sor_199, 1.99)
    c_next_SOR!(c_new_sor_193, 1.93)
    c_next_SOR!(c_new_sor_185, 1.85)
    delta_sor_199 = delta(c_old_sor_199, c_new_sor_199)
    delta_sor_193 = delta(c_old_sor_193, c_new_sor_193)
    delta_sor_185 = delta(c_old_sor_185, c_new_sor_185)
    push!(deltas_SOR_199, delta_sor_199)
    push!(deltas_SOR_193, delta_sor_193)
    push!(deltas_SOR_185, delta_sor_185)

    if delta_j < tol && delta_gs < tol && delta_sor_199 < tol && delta_sor_193 < tol && delta_sor_185 < tol
        break
    end
end

In [ ]:
plot(deltas_JACOBI, title="Convergence of methods", xlabel=L"k", ylabel=L"\delta", yscale=:log10, label="Jacobi", dpi=150)
plot!(deltas_GAUSS_SEIDEL, label="Gauss-Seidel")
plot!(deltas_SOR_199, label="SOR ω=1.99")
plot!(deltas_SOR_193, label="SOR ω=1.93")
plot!(deltas_SOR_185, label="SOR ω=1.85")
# Plot horizontal dotted line at tol
hline!([tol], linestyle=:dot, label=L"\delta = 10^{-6}")
display(current())
savefig("plots/diffusion_convergence.png")

## 1.6 J

In [ ]:
omegas_stage_1 = 1.5:0.05:1.90
omegas_stage_2 = 1.90:0.0001:1.99
omegas = vcat(omegas_stage_1, omegas_stage_2)

In [ ]:
k_converge_stage_1 = [get_iteration_count_SOR(c_0, omega, tol) for omega in omegas_stage_1]
k_converge_stage_2 = [get_iteration_count_SOR(c_0, omega, tol) for omega in omegas_stage_2]
k_converge = vcat(k_converge_stage_1, k_converge_stage_2)

plot_omega = plot(omegas, k_converge, title="SOR Iteration Count vs Omega", xlabel=L"\omega", ylabel="Iteration Count", label=L"k_{converge}", dpi=150)
# Plot vertical line at optimal omega
k_min = minimum(k_converge)
optimal_omega = omegas[findfirst(==(k_min), k_converge)]
vline!(plot_omega, [optimal_omega], linestyle=:dot, label=L"\omega_{opt} = %$optimal_omega")
display(plot_omega)
savefig("plots/sor_optimal_omega.png")

In [ ]:
plot(plot_omega, xlims=(1.85, 1.97), ylims=(0, 500))

## 1.6 K

In [ ]:
mask_sq = zeros(Bool, N, N)
mask_sq[20:30, 20:30] .= true

mask_triangle_0 = zeros(Bool, N, N)
mask_triangle_1 = zeros(Bool, N, N)
mask_triangle_2 = zeros(Bool, N, N)
mask_triangle_3 = zeros(Bool, N, N)
for i in 1:N
    mask_triangle_0[i, 1:max(i-1, 1)] .= true
    mask_triangle_1[max(1 + Int(N / 2 - i), 1):min(Int(N / 2) + i - 1, N), max(N - i, 1)] .= true
    mask_triangle_2[max(1 + Int(N / 2 - i), 1):min(Int(N / 2) + i - 1, N), max(Int(N/2)  - i, 1)] .= true
    mask_triangle_3[max(1 + Int(N / 2 - i), fld(2N, 5)):min(Int(N / 2) + i - 1, N-fld(2N, 5)), max(Int(N / 2) + 1 - i, fld(2N, 5))] .= true
end


plot(
    heatmap(mask_sq', title="Mask square"; heatmap_kwargs...),
    heatmap(mask_triangle_0', title="Mask triangle 0"; heatmap_kwargs...),
    heatmap(mask_triangle_1', title="Mask triangle 1"; heatmap_kwargs...),
    heatmap(mask_triangle_2', title="Mask triangle 2"; heatmap_kwargs...),
    heatmap(mask_triangle_3', title="Mask triangle 3"; heatmap_kwargs...),
    dpi=150,
    size=(1200, 800),
)


In [ ]:
c_sink_sq, _deltas = solve_until_tol(c_next_SOR_sink!, c_0, tol, 10_000, 1.9, mask_sq)
c_sink_tri_0, _deltas = solve_until_tol(c_next_SOR_sink!, c_0, tol, 10_000, 1.9, mask_triangle_0)
c_sink_tri_1, _deltas = solve_until_tol(c_next_SOR_sink!, c_0, tol, 10_000, 1.9, mask_triangle_1)
c_sink_tri_2, _deltas = solve_until_tol(c_next_SOR_sink!, c_0, tol, 10_000, 1.9, mask_triangle_2)
c_sink_tri_3, _deltas = solve_until_tol(c_next_SOR_sink!, c_0, tol, 10_000, 1.9, mask_triangle_3)

plot(
    heatmap(c_sink_sq', title="With sink"; heatmap_kwargs...),
    heatmap(c_sink_tri_0', title="With sink (triangle 0)"; heatmap_kwargs...),
    heatmap(c_sink_tri_1', title="With sink (triangle 1)"; heatmap_kwargs...),
    heatmap(c_sink_tri_2', title="With sink (triangle 2)"; heatmap_kwargs...),
    heatmap(c_sink_tri_3', title="With sink (triangle 3)"; heatmap_kwargs...),
    dpi=150,
    size=(1200, 800),
)

In [ ]:
plot_omegas_sinks = plot(title="SOR Iteration Count vs Omega", xlabel=L"\omega", ylabel="Iteration Count", label=L"k_{converge}", dpi=150)

omegas_stage_1_sinks = 1.5:0.01:1.95
omegas_stage_2_sinks = 1.95:0.01:1.95
omegas_sinks = vcat(omegas_stage_1_sinks, omegas_stage_2_sinks)
@showprogress for (mask, label) in zip([zeros(Bool, N, N), mask_sq, mask_triangle_0, mask_triangle_1, mask_triangle_2, mask_triangle_3], ["No sink", "Square", "Triangle 0", "Triangle 1", "Triangle 2", "Triangle 3"])
    k_converge_stage_1 = [length(solve_until_tol(c_next_SOR_sink!, c_0, tol, 10_000, omega, mask; quiet=true)[2]) for omega in omegas_stage_1_sinks]
    k_converge_stage_2 = [length(solve_until_tol(c_next_SOR_sink!, c_0, tol, 10_000, omega, mask; quiet=true)[2]) for omega in omegas_stage_2_sinks]
    k_converge = vcat(k_converge_stage_1, k_converge_stage_2)

    plot!(plot_omegas_sinks, omegas_sinks, k_converge, label=label)
    sleep(0.1)
end

display(plot_omegas_sinks)

savefig("plots/sor_optimal_omega_sinks.png")

In [ ]:
include("helpers/diffusion.jl")
c_insulator_sq, _deltas = solve_until_tol(c_next_SOR_sink_insulate!, c_0, tol, 10_000, 1.9; (insulate_mask=mask_sq))
c_insulator_tri_0, _deltas = solve_until_tol(c_next_SOR_sink_insulate!, c_0, tol, 10_000, 1.9; (insulate_mask=mask_triangle_0))
c_insulator_tri_1, _deltas = solve_until_tol(c_next_SOR_sink_insulate!, c_0, tol, 10_000, 1.9; (insulate_mask=mask_triangle_1))
c_insulator_tri_2, _deltas = solve_until_tol(c_next_SOR_sink_insulate!, c_0, tol, 10_000, 1.9; (insulate_mask=mask_triangle_2))
c_insulator_tri_3, _deltas = solve_until_tol(c_next_SOR_sink_insulate!, c_0, tol, 10_000, 1.9; (insulate_mask=mask_triangle_3))

plot(
    heatmap(c_insulator_sq', title="With insulator"; heatmap_kwargs...),
    heatmap(c_insulator_tri_0', title="With insulator (triangle 0)"; heatmap_kwargs...),
    heatmap(c_insulator_tri_1', title="With insulator (triangle 1)"; heatmap_kwargs...),
    heatmap(c_insulator_tri_2', title="With insulator (triangle 2)"; heatmap_kwargs...),
    heatmap(c_insulator_tri_3', title="With insulator (triangle 3)"; heatmap_kwargs...),
    dpi=150,
    size=(1200, 800),
)
display(current())
savefig("plots/diffusion_insulators.png")